# 🚀 LSTM加密货币价格预测 - 快速开始

> **完整的端到端LSTM价格预测教程**  
> 作者: qinshihuang166  
> 难度: ⭐⭐ (适合初学者)

---

## 📋 本笔记本内容

1. ✅ 环境设置和依赖导入
2. 📥 数据下载和探索
3. 🔧 特征工程和数据预处理
4. 🧠 LSTM模型构建
5. 🏋️ 模型训练
6. 📊 结果可视化和评估
7. 🔮 价格预测

**预计时间**: 30-45分钟

---

## 1. 环境设置和依赖导入

首先，让我们导入所有需要的库。

In [ ]:
# 基础库
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 添加项目路径
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

# 导入项目模块
from config_lstm import DataConfig, ModelConfig, TrainingConfig, PathConfig, PresetConfigs
from utils.lstm_data_processor import LSTMDataProcessor
from utils.lstm_model_builder import LSTMModelBuilder, setup_gpu
from utils.binance_client import BinanceUtility
from utils.technical_indicators import TechnicalIndicators

# TensorFlow
import tensorflow as tf
from tensorflow import keras

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.size'] = 12

print("✅ 所有依赖导入成功!")
print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 2. 配置和数据下载

我们使用快速测试配置来加快演示速度。

In [ ]:
# 应用快速测试配置
PresetConfigs.quick_test()

# 创建必要的目录
PathConfig.create_directories()

print("\n当前配置:")
print(f"  交易对: {DataConfig.SYMBOL}")
print(f"  时间间隔: {DataConfig.INTERVAL}")
print(f"  历史数据: {DataConfig.LOOKBACK_DAYS} 天")
print(f"  时间步长: {DataConfig.TIME_STEPS}")
print(f"  训练轮数: {TrainingConfig.EPOCHS}")

In [ ]:
# 下载数据
print("📥 开始下载数据...\n")

client = BinanceUtility()

# 计算开始时间
start_date = datetime.now() - timedelta(days=DataConfig.LOOKBACK_DAYS)
start_str = start_date.strftime("%d %b, %Y")

# 获取数据
df_raw = client.fetch_historical_data(
    symbol=DataConfig.SYMBOL,
    interval=DataConfig.INTERVAL,
    start_str=start_str
)

# 保存原始数据
df_raw.to_csv(DataConfig.RAW_DATA_FILE, index=False)

print(f"\n✅ 数据下载完成!")
print(f"  数据点数: {len(df_raw)}")
print(f"  时间范围: {df_raw['timestamp'].min()} 到 {df_raw['timestamp'].max()}")
print(f"\n前5行数据:")
df_raw.head()

## 3. 数据探索和可视化

让我们先看看数据长什么样。

In [ ]:
# 数据统计
print("📊 数据统计信息:\n")
print(df_raw.describe())

# 可视化价格走势
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 收盘价
axes[0].plot(df_raw['timestamp'], df_raw['close'], linewidth=1.5, color='blue')
axes[0].set_title(f'{DataConfig.SYMBOL} 收盘价走势', fontsize=14, fontweight='bold')
axes[0].set_ylabel('价格 (USDT)')
axes[0].grid(True, alpha=0.3)

# 成交量
axes[1].bar(df_raw['timestamp'], df_raw['volume'], color='orange', alpha=0.6)
axes[1].set_title('成交量', fontsize=14, fontweight='bold')
axes[1].set_xlabel('时间')
axes[1].set_ylabel('成交量')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 观察: 价格波动和成交量变化")

## 4. 特征工程

添加技术指标来增强模型的预测能力。

In [ ]:
# 创建数据处理器
processor = LSTMDataProcessor()

# 加载和清洗数据
df = processor.load_raw_data()
df = processor.clean_data(df)

# 添加技术指标
df_with_indicators = processor.add_features(df)

print(f"\n✅ 特征工程完成!")
print(f"  原始特征: {df.shape[1]} 个")
print(f"  添加后: {df_with_indicators.shape[1]} 个")
print(f"  新增: {df_with_indicators.shape[1] - df.shape[1]} 个技术指标")

In [ ]:
# 可视化一些关键技术指标
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 价格 + MA
axes[0].plot(df_with_indicators.index, df_with_indicators['close'], label='收盘价', linewidth=1.5)
axes[0].plot(df_with_indicators.index, df_with_indicators['EMA_12'], label='EMA 12', linewidth=1)
axes[0].plot(df_with_indicators.index, df_with_indicators['EMA_26'], label='EMA 26', linewidth=1)
axes[0].set_title('价格 + 移动平均线', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(df_with_indicators.index, df_with_indicators['RSI'], linewidth=1.5, color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', linewidth=1, label='超买 (70)')
axes[1].axhline(y=30, color='g', linestyle='--', linewidth=1, label='超卖 (30)')
axes[1].set_title('RSI 指标', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(df_with_indicators.index, df_with_indicators['MACD'], label='MACD', linewidth=1.5)
axes[2].plot(df_with_indicators.index, df_with_indicators['MACD_signal'], label='Signal', linewidth=1.5)
axes[2].bar(df_with_indicators.index, df_with_indicators['MACD_hist'], label='Histogram', alpha=0.3)
axes[2].set_title('MACD 指标', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 观察: 技术指标如何反映市场状态")

## 5. 数据预处理

选择特征、归一化、创建时间序列窗口。

In [ ]:
# 选择特征
df_selected = processor.select_features(df_with_indicators)

# 归一化
df_normalized = processor.normalize_data(df_selected, fit=True)

# 创建时间序列窗口
X, y = processor.create_sequences(df_normalized.values)

# 划分数据集
X_train, X_val, X_test, y_train, y_val, y_test = processor.split_data(X, y)

print(f"\n✅ 数据预处理完成!")
print(f"\n数据形状:")
print(f"  X_train: {X_train.shape} - (样本数, 时间步, 特征数)")
print(f"  y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}")
print(f"  y_val: {y_val.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")

## 6. 构建LSTM模型

创建双向LSTM模型。

In [ ]:
# 配置GPU
has_gpu = setup_gpu()

# 创建模型构建器
model_builder = LSTMModelBuilder()

# 构建模型
input_shape = (X_train.shape[1], X_train.shape[2])
model = model_builder.build_model(input_shape)

print("\n✅ 模型构建完成!")

## 7. 训练模型

使用早停和学习率衰减训练模型。

In [ ]:
# 获取回调函数
callbacks = model_builder.get_callbacks()

# 训练模型
print("\n🚀 开始训练...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=TrainingConfig.EPOCHS,
    batch_size=TrainingConfig.BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ 训练完成!")

## 8. 训练结果可视化

查看训练过程中loss的变化。

In [ ]:
# 绘制训练历史
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='训练Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='验证Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('训练和验证Loss', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='训练MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='验证MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('训练和验证MAE', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印最佳结果
best_epoch = np.argmin(history.history['val_loss']) + 1
best_val_loss = np.min(history.history['val_loss'])

print(f"\n📊 训练摘要:")
print(f"  训练轮数: {len(history.history['loss'])}")
print(f"  最佳Epoch: {best_epoch}")
print(f"  最佳验证Loss: {best_val_loss:.6f}")

## 9. 模型评估

在测试集上评估模型性能。

In [ ]:
# 测试集评估
test_results = model.evaluate(X_test, y_test, verbose=0)

print("📊 测试集性能:")
print(f"  Loss (MSE): {test_results[0]:.6f}")
print(f"  MAE: {test_results[1]:.6f}")
print(f"  RMSE: {np.sqrt(test_results[0]):.6f}")

## 10. 预测和可视化

使用模型进行预测并与实际值对比。

In [ ]:
# 在测试集上预测
y_pred = model.predict(X_test, verbose=0).flatten()

# 可视化
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 预测 vs 实际
display_points = min(200, len(y_test))
x_range = range(display_points)

axes[0].plot(x_range, y_test[:display_points], label='实际价格', linewidth=2, alpha=0.7)
axes[0].plot(x_range, y_pred[:display_points], label='预测价格', linewidth=2, alpha=0.7)
axes[0].set_xlabel('时间步')
axes[0].set_ylabel('归一化价格')
axes[0].set_title('预测 vs 实际价格', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 误差分布
errors = y_test - y_pred
axes[1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='零误差线')
axes[1].set_xlabel('预测误差')
axes[1].set_ylabel('频数')
axes[1].set_title('预测误差分布', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 统计信息
print(f"\n📊 预测统计:")
print(f"  平均误差: {np.mean(errors):.6f}")
print(f"  误差标准差: {np.std(errors):.6f}")
print(f"  最大误差: {np.max(np.abs(errors)):.6f}")

## 11. 保存模型

保存训练好的模型以供后续使用。

In [ ]:
# 保存模型
model_builder.save_model()

# 保存scaler
processor.save_scaler()

print("\n✅ 模型和预处理器已保存!")
print(f"  模型: {PathConfig.MODEL_PATH}")
print(f"  Scaler: {PathConfig.SCALER_PATH}")

## 🎉 恭喜！

你已经成功完成了LSTM价格预测的完整流程！

### 📚 下一步学习

1. **深入理解LSTM** - 阅读 `LSTM_TUTORIAL.md`
2. **探索其他笔记本**:
   - `02_特征工程_技术指标.ipynb` - 深入理解技术指标
   - `03_LSTM模型构建.ipynb` - 理解模型架构
   - `06_参数调优指南.ipynb` - 学习调优技巧
3. **实战应用**:
   - 尝试不同的交易对
   - 调整模型参数
   - 添加自定义特征

### 💡 关键要点

- ✅ LSTM擅长处理时间序列数据
- ✅ 技术指标能提供有价值的市场信息
- ✅ 数据归一化对模型训练很重要
- ✅ 使用回调函数防止过拟合
- ✅ 在测试集上评估模型真实性能

---

**作者**: qinshihuang166  
**项目**: [binance-lstm-prediction](https://github.com/qinshihuang166)

⚠️ **免责声明**: 本教程仅供学习，不构成投资建议。
